## Info
based on notebook3

## Load packages and the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

2025-09-23 15:21:12.262907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-23 15:21:13.043092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-23 15:21:13.043140: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-23 15:21:13.047947: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-23 15:21:13.520605: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-23 15:21:13.524342: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

### Copy weights

In [2]:
ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Look at the model

In [25]:
model.get_config().keys()
model.get_config()['layers'][1]

/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


{'module': 'qkeras.qlayers',
 'class_name': 'QDense',
 'config': {'name': 'q_dense',
  'trainable': True,
  'dtype': 'float32',
  'units': 29,
  'activation': {'module': 'qkeras.quantizers',
   'class_name': 'quantized_relu',
   'config': {'bits': 10,
    'integer': 6,
    'use_sigmoid': 0,
    'negative_slope': 0.0,
    'use_stochastic_rounding': False,
    'relu_upper_bound': None,
    'qnoise_factor': 1.0},
   'registered_name': 'quantized_relu'},
  'use_bias': True,
  'kernel_initializer': {'module': 'qkeras.qlayers',
   'class_name': 'QInitializer',
   'config': {'initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'use_scale': True,
    'quantizer': {'module': 'qkeras.quantizers',
     'class_name': 'quantized_bits',
     'config': {'bits': 6,
      'integer': 2,
      'symmetric': 0,
      'alpha': 1,
      'keep_negative': True,
      'use_stochastic_rounding': False,
      'qnoise_